In [1]:
import tensorflow as tf
import pandas as pd
import urllib.request

/Users/a59347/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
url = "https://raw.githubusercontent.com/vincentarelbundock/Rdatasets/master/csv/datasets/mtcars.csv"
file_name = "data.csv"
urllib.request.urlretrieve(url, file_name)

('data.csv', <http.client.HTTPMessage at 0x1814229e80>)

In [3]:
df = pd.read_csv(url)
df.head()

,Unnamed: 0,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [6]:
def input_fn(data_file, num_epochs, shuffle, batch_size):
    
    _COLUMN_DEFAULTS = [
        tf.constant([''], dtype=tf.string),    #0: car 
        tf.constant([0], dtype=tf.float32),    #1: mpg 
        tf.constant([0], dtype=tf.int32),    #2: cyl
        tf.constant([0], dtype=tf.float32),   #3: disp
        tf.constant([0], dtype=tf.int32),   #4: hp
        tf.constant([0], dtype=tf.float32), #5. drat
        tf.constant([0], dtype=tf.float32), # wt
        tf.constant([0], dtype=tf.float32), # qsec
        tf.constant([0], dtype=tf.int32), # vs
        tf.constant([0], dtype=tf.int32), # am
        tf.constant([0], dtype=tf.int32), # gear
        tf.constant([0], dtype=tf.int32), # carb
    ]

    _COLUMNS = [
        'car', 'mpg', 'cyl', 'disp', 'hp', 'drat',
        'wt', 'qsec', 'vs', 'am', 'gear', 'carb'
    ]

    def parse_csv(v):
        columns = tf.decode_csv(v, record_defaults=_COLUMN_DEFAULTS)

        '''
        record_defaults: A list of Tensor objects with types from: 
        float32, int32, int64, string. One tensor per column of the input record, 
        with either a scalar default value for that column or empty if the column is required.
        '''
        features = dict(zip(_COLUMNS, columns))
        labels = features.pop('mpg')
        mpg_threshold = 21
        return features, tf.greater(labels, mpg_threshold)

    dataset = tf.data.TextLineDataset(data_file)
    dataset = dataset.map(parse_csv, num_parallel_calls=5)

    # We call repeat after shuffling, rather than before, to prevent separate
    # epochs from blending together.
    dataset = dataset.repeat(num_epochs)
    dataset = dataset.batch(batch_size)

    iterator = dataset.make_one_shot_iterator()
    features, labels = iterator.get_next()
    for feature in features:
        print(feature)
        print(features[feature])
    print('-'*10)
    print(labels)
    print('-'*10)

    return features, labels

print(input_fn("data.csv", 100, True, 2))

car
Tensor("IteratorGetNext:1", shape=(?,), dtype=string)
cyl
Tensor("IteratorGetNext:3", shape=(?,), dtype=int32)
disp
Tensor("IteratorGetNext:4", shape=(?,), dtype=float32)
hp
Tensor("IteratorGetNext:7", shape=(?,), dtype=int32)
drat
Tensor("IteratorGetNext:5", shape=(?,), dtype=float32)
wt
Tensor("IteratorGetNext:10", shape=(?,), dtype=float32)
qsec
Tensor("IteratorGetNext:8", shape=(?,), dtype=float32)
vs
Tensor("IteratorGetNext:9", shape=(?,), dtype=int32)
am
Tensor("IteratorGetNext:0", shape=(?,), dtype=int32)
gear
Tensor("IteratorGetNext:6", shape=(?,), dtype=int32)
carb
Tensor("IteratorGetNext:2", shape=(?,), dtype=int32)
----------
Tensor("IteratorGetNext:11", shape=(?,), dtype=bool)
----------
({'car': <tf.Tensor 'IteratorGetNext:1' shape=(?,) dtype=string>, 'cyl': <tf.Tensor 'IteratorGetNext:3' shape=(?,) dtype=int32>, 'disp': <tf.Tensor 'IteratorGetNext:4' shape=(?,) dtype=float32>, 'hp': <tf.Tensor 'IteratorGetNext:7' shape=(?,) dtype=int32>, 'drat': <tf.Tensor 'IteratorGe

In [40]:
# Iterator is a bit mystery to me at this moment, need to do some homework.
# https://www.tensorflow.org/api_docs/python/tf/data/Iterator